Step 0: clone the ds repo

In [ ]:
!git clone https://github.com/neonsecret/t5-inq-rep.git 
!mv t5-inq-rep/* .
!rm -rf t5-inq-rep

Cloning into 't5-inq-rep'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 57 (delta 11), reused 54 (delta 8), pack-reused 0
Unpacking objects: 100% (57/57), done.


In [ ]:
!pip install transformers==3.0.0 nltk nlp==0.2.0 language_tool_python nlpaug gensim>=4.1.2

import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Step 1: finetuning from the dataset

In [ ]:
#to prepare data use:
!python prepare_data.py --task qg --model_type t5 --dataset_path data/inquisitive_qg/ --qg_format plain_text --max_source_length 512 --max_target_length 32 --train_file_name train_data_inquisitive_qg.pt --valid_file_name valid_data_inquisitive_qg.pt

Traceback (most recent call last):
  File "prepare_data.py", line 8, in <module>
    from transformers import T5Tokenizer, BartTokenizer, HfArgumentParser
ModuleNotFoundError: No module named 'transformers'


In [ ]:
#to fine-tune using a command, use:
!python run_qg.py --model_name_or_path t5-small --model_type t5 --tokenizer_name_or_path t5_qg_tokenizer --output_dir t5-small-qg-inqds --train_file_path data/train_data_inquisitive_qg.pt --valid_file_path data/valid_data_inquisitive_qg.pt --per_device_train_batch_size 16 --per_device_eval_batch_size 16 --gradient_accumulation_steps 16 --learning_rate 1e-4 --num_train_epochs 15 --seed 42 --do_train --do_eval --logging_steps 100

In [ ]:
#to eval with a command, use:
!python eval.py --model_name_or_path t5-small-qg-inqds --valid_file_path t5-small-qg-inqds\checkpoint-1500\pytorch_model.bin --model_type t5 --num_beams 4 --max_decoding_length 32 --output_path result_metrics.txt --tokenizer_name_or_path t5_qg_tokenizer --valid_file_path data\valid_data_qg_hl_t5.pt
#dont forget to replace 1500 with your checkpoint number

In [ ]:

#to do the metrics, do:
!nlg-eval --hypothesis=result_metrics.txt --references=data/references.txt --no-skipthoughts --no-glove

Step two: the loop

In [ ]:
#the imports
import nlp
import torch
import time
import random
import language_tool_python
import threading

from torch import nn
from typing import Any, Dict, Union
from transformers import Trainer as HFTrainer
from transformers import (
    AutoModelForSeq2SeqLM,
    T5Tokenizer,
)
from torch.utils.data import DataLoader
from ipywidgets import interact
from IPython.display import display

import pandas as pd
import numpy as np
import ipywidgets as widgets
import nlpaug.augmenter.word as naw

In [ ]:
def generate(text, **kwargs):  #standard huggingface generate method
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(**inputs, num_beams=5, **kwargs)
    return postprocess_model_out(tokenizer.decode(hypotheses[0], skip_special_tokens=True))


def triplet_loss(x, y, z):  
    return torch.nn.TripletMarginLoss(
        margin=1.0, p=2.0, eps=1e-06, swap=False,
        size_average=None, reduce=None, reduction='mean'
    )(x, y, z)

def postprocess_model_out(out):
    out = out.replace("hl_token", "").replace("hl", "")
    out = tool.correct(out)
    if random.random() < 0.3: #random hardcoded chance of augmentation
      out = aug.augment(out)
    return out

def filter_qg(example):
    return example['task'] == 'qg'

In [ ]:
#the loop GUI

#preparations
tokenizer = T5Tokenizer.from_pretrained("t5-small", cache_dir=None)
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small", cache_dir=None)
model.resize_token_embeddings(len(tokenizer))

#added squad multitask for..educational purposes
train_dataset = nlp.load_dataset("data/inquisitive_qg/", name="plain_text", split=nlp.Split.TRAIN)
# train_dataset = nlp.load_dataset("data/squad_multitask/", name="highlight_qg_format", split=nlp.Split.TRAIN)

train_dataset = train_dataset.filter(filter_qg)

tool = language_tool_python.LanguageTool('en-US')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
batch_size = 1  #for comfy finetuning
log_freq = 1    #print every
epochs = 3


can_go = False
generate_questions_N = 3

memory_chance = .5

model.train()
losses = []

global_chosen_sentence = ""
memory = []

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

epoch_n = 0

aug = naw.SynonymAug(aug_src='wordnet')


it = iter(train_dataloader)
data = next(it)
inner_value = "<h1>" + data["source_text"][0] + "</h1>" #converting to html
inner_value = inner_value.replace("{hl_token}", "<font color='maroon'>", 1).replace("{hl_token}", "</font>")   #coloring the query termin
title_widget = widgets.HTML(
    value = inner_value
)
display(title_widget)


sentence_widget = widgets.SelectMultiple(  
    options=[generate(data["source_text"][0], max_length=32)+"?" for _ in range(generate_questions_N)],  #generating N questions
    rows=10, 
    description='Sentences',
    disabled=False,
    layout= widgets.Layout(width='50%')
)
display(sentence_widget)

label = widgets.Text(description="or enter:", value='')
display(label)

button = widgets.Button(description="Choose", disabled=False, button_style="success")
output = widgets.Output()
display(button, output)

def annotate(_):  # the main training method
    global model, memory, losses, data, it, epoch_n, memory_chance
    with output:
      if label.value != "":  # if no input
        used_sentence = label.value
      else:
        used_sentence = sentence_widget.value[0]
      print("using " + "'" +used_sentence + "'")
      x = tokenizer(data["source_text"][0], return_tensors='pt', padding=True).to(model.device) 
      y = tokenizer(used_sentence, return_tensors='pt', padding=True).to(model.device)

      memory.append((x, y))  #adding to memory so we can refinetune on it later

      y.input_ids[y.input_ids == 0] = -100

      loss = model(
          input_ids=x.input_ids,
          attention_mask=x.attention_mask,
          labels=y.input_ids,
          decoder_attention_mask=y.attention_mask,
          # return_dict=True
      )[0]

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      losses.append(loss.item())
      epoch_n+=1
      print("epoch: {}, loss: {}".format(epoch_n, loss.item()))

      data = next(it)
      inner_value = "<h1>" + data["source_text"][0] + "</h1>"  #same html and coloring thing
      inner_value = inner_value.replace("{hl_token}", "<font color='maroon'>", 1).replace("{hl_token}", "</font>")
      title_widget.value = inner_value

      label.value = ""


      sentence_widget.options = tuple([generate(data["source_text"][0], max_length=32)+"?" for _ in range(generate_questions_N)])  #generating the N questions

      
      if random.random() < memory_chance:
        # finetune on a used item
        x, y = random.choice(memory)
        logits = model(
        input_ids=x.input_ids,
        attention_mask=x.attention_mask,
        labels=y.input_ids,
        decoder_attention_mask=y.attention_mask,
        # return_dict=True
        )[1]  #extracting the raw model outputs, not the loss

        logits = torch.topk(logits, 1)[0].squeeze(-1)  #extracting the answers from logits

        neg = random.choice(memory)[1]  #the 3rd component for the triple loss

        sh1 = logits.shape[-1]
        sh2 = y.input_ids.shape[-1]
        sh3 = neg.input_ids.shape[-1]

        sh = min([sh1, sh2, sh3])  #lining up the shapes

        logits = logits[:, :sh]  
        y.input_ids = y.input_ids[:, :sh]   
        neg.input_ids = neg.input_ids[:, :sh]

        # print(sh, logits.shape, y.input_ids.shape, neg.input_ids.shape)

        loss = triplet_loss(logits, y.input_ids, neg.input_ids)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print("random memory triplet loss: {}".format(round(float(loss), 2)))



button.on_click(annotate)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


...

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1500: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


SelectMultiple(description='Sentences', layout=Layout(width='50%'), options=("Dunking's poison pill rights and…

Text(value='', description='or enter:')

Button(button_style='success', description='Choose', style=ButtonStyle())

Output()